## Impulse MACD [LazyBear]

<https://www.tradingview.com/script/qt6xLfLi-Impulse-MACD-LazyBear/>

>Developed by [@edyatl](https://github.com/edyatl) March 2023 <edyatl@yandex.ru>

In [1]:
# Load Jupyter extension for auto correction coding style based on Black Lib
%load_ext nb_black

<IPython.core.display.Javascript object>

### Original Indicator code

```python
//
// @author LazyBear 
// 
// List of my public indicators: http://bit.ly/1LQaPK8 
// List of my app-store indicators: http://blog.tradingview.com/?p=970 
//
//
study("Impulse MACD [LazyBear]", shorttitle="IMACD_LB", overlay=false)
lengthMA = input(34)
lengthSignal = input(9)
calc_smma(src, len) =>
	smma=na(smma[1]) ? sma(src, len) : (smma[1] * (len - 1) + src) / len
	smma

calc_zlema(src, length) =>
	ema1=ema(src, length)
	ema2=ema(ema1, length)
	d=ema1-ema2
	ema1+d

src=hlc3
hi=calc_smma(high, lengthMA)
lo=calc_smma(low, lengthMA)
mi=calc_zlema(src, lengthMA) 

md=(mi>hi)? (mi-hi) : (mi<lo) ? (mi - lo) : 0
sb=sma(md, lengthSignal)
sh=md-sb
mdc=src>mi?src>hi?lime:green:src<lo?red:orange
plot(0, color=gray, linewidth=1, title="MidLine")
plot(md, color=mdc, linewidth=2, title="ImpulseMACD", style=histogram)
plot(sh, color=blue, linewidth=2, title="ImpulseHisto", style=histogram)
plot(sb, color=maroon, linewidth=2, title="ImpulseMACDCDSignal")

ebc=input(false, title="Enable bar colors")
barcolor(ebc?mdc:na)
```

In [2]:
# Standard imports
import pandas as pd
import numpy as np

# import matplotlib.pyplot as plt
# import seaborn as sns
import talib as tl

import os
from os import environ as env
from dotenv import load_dotenv
from binance import Client, ThreadedWebsocketManager, ThreadedDepthCacheManager

# Nicest style for plots
# sns.set(style="ticks")

<IPython.core.display.Javascript object>

In [3]:
# Install a pip package in the current Jupyter kernel
# import sys

# !{sys.executable} -m pip install -U python-dotenv

<IPython.core.display.Javascript object>

In [4]:
project_dotenv = os.path.join(os.path.abspath(""), ".env")
if os.path.exists(project_dotenv):
    load_dotenv(project_dotenv)

<IPython.core.display.Javascript object>

In [5]:
api_key, api_secret = env.get("ENV_API_KEY"), env.get("ENV_SECRET_KEY")
client = Client(api_key, api_secret)

<IPython.core.display.Javascript object>

In [6]:
klines = client.get_klines(symbol="ATOMUSDT", interval=Client.KLINE_INTERVAL_15MINUTE)

short_col_names = [
    "open_time",
    "open",
    "high",
    "low",
    "close",
    "volume",
    "close_time",
    "qav",
    "num_trades",
    "taker_base_vol",
    "taker_quote_vol",
    "ignore",
]

<IPython.core.display.Javascript object>

In [7]:
data = pd.DataFrame(klines, columns=short_col_names)
data["open_time"] = pd.to_datetime(data["open_time"], unit="ms")
data["close_time"] = pd.to_datetime(data["close_time"], unit="ms")
data.tail(5)

,open_time,open,high,low,close,volume,close_time,qav,num_trades,taker_base_vol,taker_quote_vol,ignore
495,2023-03-05 23:30:00,12.01000000,12.03500000,12.00000000,12.00000000,5161.56000000,2023-03-05 23:44:59.999,62048.65273000,250,2563.87000000,30819.09144000,0
496,2023-03-05 23:45:00,12.00300000,12.03900000,11.99900000,12.01900000,2975.20000000,2023-03-05 23:59:59.999,35775.37734000,175,1091.16000000,13119.13223000,0
497,2023-03-06 00:00:00,12.01900000,12.04400000,12.00000000,12.00800000,4168.94000000,2023-03-06 00:14:59.999,50134.19344000,343,1998.63000000,24038.15067000,0
498,2023-03-06 00:15:00,12.00400000,12.03600000,11.95800000,12.03600000,6773.73000000,2023-03-06 00:29:59.999,81238.93126000,408,3281.02000000,39342.03309000,0
499,2023-03-06 00:30:00,12.03200000,12.04500000,12.02700000,12.03900000,4405.79000000,2023-03-06 00:44:59.999,53032.77966000,222,2368.11000000,28503.89073000,0


<IPython.core.display.Javascript object>

### Inputs

In [8]:
lengthMA: int = 34  # input(34)
lengthSignal: int = 9  # input(9)

<IPython.core.display.Javascript object>

In [9]:
def calc_smma(src: np.ndarray, length: int) -> np.ndarray:
    """
    Calculate Smoothed Moving Average (SMMA) for a given numpy array `src` with a specified `length`.

    :param src: A numpy ndarray of shape (n,) containing the input values of float64 dtype.
    :param length: An integer representing the length of the SMMA window.
    :return: A numpy ndarray of the same shape as `src` containing the SMMA values.
    """
    smma = np.full_like(src, fill_value=np.nan)
    sma = tl.SMA(src, length)

    for i in range(1, len(src)):
        smma[i] = (
            sma[i]
            if np.isnan(smma[i - 1])
            else (smma[i - 1] * (length - 1) + src[i]) / length
        )

    return smma

<IPython.core.display.Javascript object>

In [10]:
def calc_zlema(src: np.ndarray, length: int) -> np.ndarray:
    """
    Calculates the zero-lag exponential moving average (ZLEMA) of the given price series.

    :param src: The input price series of float64 dtype to calculate the ZLEMA for.
    :param length: int The number of bars to use for the calculation of the ZLEMA.
    :return: A numpy ndarray of ZLEMA values for the input price series.
    """
    ema1 = tl.EMA(src, length)
    ema2 = tl.EMA(ema1, length)
    d = ema1 - ema2
    return ema1 + d

<IPython.core.display.Javascript object>

In [11]:
src = (
    data["high"].to_numpy(dtype=np.double)
    + data["low"].to_numpy(dtype=np.double)
    + data["close"].to_numpy(dtype=np.double)
) / 3
hi = calc_smma(data["high"].to_numpy(dtype=np.double), lengthMA)
lo = calc_smma(data["low"].to_numpy(dtype=np.double), lengthMA)
mi = calc_zlema(src, lengthMA)
# hi

<IPython.core.display.Javascript object>

In [12]:
md = np.full_like(mi, fill_value=np.nan)

conditions = [mi > hi, mi < lo]
choices = [mi - hi, mi - lo]

md = np.select(conditions, choices, default=0)

sb = tl.SMA(md, lengthSignal)
sh = md - sb
# md.dtype
pd.Series(md).tail(10)

490    0.010724
491    0.000385
492    0.000000
493    0.000000
494    0.000000
495    0.000000
496   -0.005292
497   -0.012364
498   -0.018208
499   -0.022313
dtype: float64

<IPython.core.display.Javascript object>

In [13]:
res = pd.DataFrame(
    {
        "open_time": data["open_time"],
        "ImpulseMACD": md,
        "ImpulseHisto": sh,
        "ImpulseMACDCDSignal": sb,
    }
)
res.tail(20)

,open_time,ImpulseMACD,ImpulseHisto,ImpulseMACDCDSignal
480,2023-03-05 19:45:00,0.050964,-0.005257,0.056221
481,2023-03-05 20:00:00,0.048297,-0.006845,0.055141
482,2023-03-05 20:15:00,0.046851,-0.007130,0.053981
483,2023-03-05 20:30:00,0.044761,-0.007867,0.052627
484,2023-03-05 20:45:00,0.042759,-0.008368,0.051127
485,2023-03-05 21:00:00,0.041835,-0.007596,0.049431
486,2023-03-05 21:15:00,0.038502,-0.008791,0.047293
487,2023-03-05 21:30:00,0.031019,-0.013373,0.044392
488,2023-03-05 21:45:00,0.025113,-0.016009,0.041122
489,2023-03-05 22:00:00,0.018453,-0.019057,0.037510


<IPython.core.display.Javascript object>